# Pendientes

Nada

# Gpu

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

# Set_Up

In [ ]:
symbol = "BTCUSD"
root_data = f'/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

direction = 'Short'
direction_number = -1

symbol = 'BTCUSD'
strategy = 'Kalman'
time_frame = 'M5'

trade_evolution = 'st_Max'
result_field = 'st_PnL'


# Libraries

In [ ]:
!pip install ta-lib
import talib as ta
print(ta.__version__)

In [ ]:
import numpy as np
import pandas as pd
import os
import joblib
import math

from tqdm.auto import tqdm

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from xgboost import XGBClassifier, XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, r2_score

import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit

import sys
sys.path.append("..")

from __future__ import annotations
from typing import Tuple, List, Optional, Dict, Any

from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import (roc_auc_score, f1_score, accuracy_score, log_loss, r2_score)

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Support Functions

In [ ]:
def kalman_line(source, kalman_length: int, smooth: int):

    n = len(source)
    kf_c = np.empty(n)            # núcleo del filtro
    velo_c = np.zeros(n)          # componente de velocidad

    sqrt_term   = np.sqrt(kalman_length / 10000.0 * 2.0)
    length_term = kalman_length / 10000.0

    # --------- inicialización (mismo efecto que `var` en Pine) ----------
    kf_c[0]   = source.iloc[0]    # nz(kf_c[1], source) para la primera barra
    velo_c[0] = 0.0

    # ------------------- bucle recursivo -------------------------------
    for i in range(1, n):
        prev_kf = kf_c[i-1] if not np.isnan(kf_c[i-1]) else source.iloc[i]
        dk      = source.iloc[i] - prev_kf
        smooth_c = prev_kf + dk * sqrt_term          # parte "suave"
        velo_c[i] = velo_c[i-1] + length_term * dk   # acumulamos velocidad
        kf_c[i]   = smooth_c + velo_c[i]             # estimación final

    # -------------------- EMA final (ta.ema) ----------------------------
    kf_c_series = pd.Series(kf_c, index=source.index)
    kalman_line = kf_c_series.ewm(span=smooth, adjust=False).mean()
    return kalman_line

In [ ]:
def slope(src: pd.Series,
          length_kal: int,
          smooth_kal: int,
          slopeLen: int,
          offset: int) -> pd.DataFrame:

    n = len(src)
    kf_state = np.full(n, np.nan)
    kf_velo  = np.zeros(n)
    sqrt_factor = np.sqrt(length_kal / 10000.0 * 2.0)
    vel_factor  = length_kal / 10000.0

    for i in range(n):
        if i == 0:
            prev_state = src.iloc[0]
            prev_velo  = 0.0
        else:
            prev_state = kf_state[i-1] if not np.isnan(kf_state[i-1]) else src.iloc[i]
            prev_velo  = kf_velo[i-1]

        dk = src.iloc[i] - prev_state
        smooth = prev_state + dk * sqrt_factor
        kf_velo[i]  = prev_velo + vel_factor * dk
        kf_state[i] = smooth + kf_velo[i]

    # 2) EMA smoothing --------------------------------------------------
    kal = pd.Series(kf_state, index=src.index).ewm(span=smooth_kal, adjust=False).mean()

    # 3) Slope/divergence -----------------------------------------------
    validLen = max(slopeLen, 1)
    slope_div = kal.diff(validLen) / validLen
    slope_signal = (slope_div > slope_div.shift(1)).astype(int)

    # 4) Angle in degrees -----------------------------------------------
    price_change = kal - kal.shift(validLen)
    slope_angle = np.degrees(np.arctan(price_change))
    slope_angle_signal = (slope_angle > slope_angle.shift(1)).astype(int)

    # 5) Linear regression prediction ----------------------------------
    def _linreg(y):
        x = np.arange(len(y))
        m, b = np.polyfit(x, y, 1)
        return b + m * (len(y)-1)

    slope_lin_reg = kal.rolling(window=slopeLen).apply(_linreg, raw=False)
    slope_lin_reg = slope_lin_reg.shift(-offset)  # apply Pine-style offset
    slope_lin_reg_signal = (slope_lin_reg > slope_lin_reg.shift(1)).astype(int)

    # 6) Pack results ---------------------------------------------------
    return pd.DataFrame({
        'slope_div':            slope_div,
        'slope_signal':         slope_signal,
        'slope_angle':          slope_angle,
        'slope_angle_signal':   slope_angle_signal,
        'slope_lin_reg':        slope_lin_reg,
        'slope_lin_reg_signal': slope_lin_reg_signal
    })

In [ ]:

def scale_features_data(features):
    # Standardise the input data (X)
    scaler = StandardScaler()

    # Split the dataset into training and testing sets as 80:20
    train_data = features.iloc[:(int(len(features) * 0.8))]
    test_data = features.iloc[(int(len(features) * 0.8)):]

    # Scale the training and testing sets
    X_train = pd.DataFrame(data=scaler.fit_transform(
        train_data), columns=features.columns, index=train_data.index)
    X_test = pd.DataFrame(data=scaler.transform(test_data),
                          columns=features.columns, index=test_data.index)

    return X_train, X_test

# Function for creating target variable
def target_var(data, window_size=20):
    target = pd.DataFrame(index=data.index)
    target['signal'] = data.Close.pct_change(window_size).shift(-window_size)

    target.dropna(inplace=True)
    target['signal'] = np.where(target['signal'] > 0, 1, -1)
    return target

# Function to split and scale the data
def train_test_split(features, target, split_proportion=0.8):

    split_index = int(len(features) * split_proportion)

    X_train = features.iloc[:split_index]
    X_test = features.iloc[split_index:]
    y_train = target.iloc[:split_index]
    y_test = target.iloc[split_index:]

    scaler = StandardScaler()
    X_train = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

    return X_train, X_test, y_train, y_test, scaler


In [ ]:
def trade_analytics(trades):
    analytics = pd.DataFrame(index=['Strategy'])
    analytics['Total PnL'] = round(trades.PnL.sum(),2)
    analytics['Total Trades'] = len(trades.loc[trades.Position!=0])
    analytics['Number of Winners'] = len(trades.loc[trades.PnL>0])
    analytics['Number of Losers'] = len(trades.loc[trades.PnL<=0])
    analytics['Win (%)'] = round(100*analytics['Number of Winners']/analytics['Total Trades'],2)
    analytics['Loss (%)'] = round(100*analytics['Number of Losers']/analytics['Total Trades'],2)
    analytics['Average Profit of Winning Trade'] = round(trades.loc[trades.PnL>0].PnL.mean(),2)
    analytics['Average Loss of Losing Trade'] = round(np.abs(trades.loc[trades.PnL<=0].PnL.mean()),2)
    trades['Entry Date'] = pd.to_datetime(trades['Entry Date'])
    trades['Exit Date'] = pd.to_datetime(trades['Exit Date'])
    holding_period = trades['Exit Date'] - trades['Entry Date']
    analytics['Average Holding Time'] = holding_period.mean()
    analytics['Profit Factor'] = round((analytics['Win (%)']/100*analytics['Average Profit of Winning Trade'])/(analytics['Loss (%)']/100*analytics['Average Loss of Losing Trade']),2)
    return analytics.T

def performance_metrics(data):
    data.set_index('Date', inplace=True)
    performance_metrics = pd.DataFrame(index=['Strategy'])
    data['Strategy Returns'] = data.signal.shift(1) * data.Close.pct_change()
    data['Cumulative Returns'] = (data['Strategy Returns'] + 1.0).cumprod()
    data['Cumulative Benchmark Returns'] = (data['Close'].pct_change() +1).cumprod()
    data['Cumulative Returns'].plot(figsize=(15, 7), label='Strategy Returns')
    data['Cumulative Benchmark Returns'].plot(label='Benchmark Returns')
    plt.title('Equity Curve', fontsize=14)
    plt.ylabel('Cumulative Returns', fontsize = 12)
    plt.xlabel('Date', fontsize = 12)
    plt.legend()
    plt.show()
    days = len(data['Cumulative Returns'])
    performance_metrics['CAGR'] = "{0:.2f}%".format(
        (data['Cumulative Returns'].iloc[-1]**(252/days)-1)*100)
    performance_metrics['Annualised Volatility'] = "{0:.2f}%".format(
        data['Strategy Returns'].std()*np.sqrt(252) * 100)
    risk_free_rate = 0.02/252
    performance_metrics['Sharpe Ratio'] = round(np.sqrt(252)*(np.mean(data['Strategy Returns']) -
                                                        (risk_free_rate))/np.std(data['Strategy Returns']),2)
    data['Peak'] = data['Cumulative Returns'].cummax()
    data['Drawdown'] = ((data['Cumulative Returns'] - data['Peak'])/data['Peak'])
    performance_metrics['Maximum Drawdown'] =  "{0:.2f}%".format((data['Drawdown'].min())*100)
    plt.figure(figsize=(15, 7))
    plt.title('Drawdowns', fontsize=14)
    plt.ylabel('Drawdown', fontsize=12)
    plt.xlabel('Date', fontsize=12)
    plt.plot(data['Drawdown'], color='red')
    plt.fill_between(data['Drawdown'].index, data['Drawdown'].values, color='red')
    plt.show()
    print(performance_metrics.T)

In [ ]:
def predict_signals(X_test, aapl_test_prices_ts, model, scaler):
    # Initialise current position
    current_pos = 0

    # Initialise count of holding days
    hold_days = 0

    # Iterate through the rows of test data
    for dt, row in X_test.iterrows():
        # Check if there is no position or holding period reaches 20 days
        if current_pos == 0 or hold_days == 20:
            # Prepare test data for prediction
            test = pd.DataFrame(data=scaler.transform(
                row.values.reshape(1, -1)), columns=X_test.columns)

            # Generate signal based on test data
            signal = model.predict(test)[-1]

            # Update current position
            current_pos = signal

            # Update predicted and actual labels for the current date
            aapl_test_prices_ts.loc[dt, 'signal'] = current_pos

            # Reset holding days counter
            hold_days = 0
        elif current_pos != 0:
            # If there is an existing position, increment holding days counter
            hold_days += 1

    # Forward fill the last observed value for 'y_pred'
    aapl_test_prices_ts['signal'].ffill(inplace=True)

    return aapl_test_prices_ts

In [ ]:
def calculate_portfolio_returns_csmom(monthly_returns, portfolio='long-short', lookback_months=12):
    stock_monthly_returns = pd.DataFrame()

    # Loop through each month after the lookback period
    for i in range(lookback_months, len(monthly_returns)):
        # Select the subset of monthly returns for the current lookback period
        # Store the historical and holding monthly returns data and store in 'returns'
        returns = monthly_returns[i - lookback_months:i + 1]

        # Store the historical in 'trailing_returns'
        trailing_returns = returns[:lookback_months]

        # Extract the starting, ending, and holding months from the subset
        starting_month = str(returns.index[0])[:7]
        ending_month = str(returns.index[-2])[:7]
        holding_month = str(returns.index[-1])[:7]

        # Set returns data as the transposed scaled trailing returns
        returns_data = trailing_returns.T

        # Initialize the number of clusters and maximum number of stocks per cluster
        num_clusters = 1
        max_stocks_per_cluster = 10

        # Perform hierarchical clustering using 'ward' linkage method
        linkage_matrix = linkage(trailing_returns.T, method='ward')

        # Assign cluster labels to stocks, ensuring each cluster has at most 10 stocks
        clusters = fcluster(linkage_matrix, num_clusters, criterion='maxclust')

        # Assign the cluster labels to the original returns data
        returns_data['Cluster'] = clusters

        # Adjust clusters until each cluster meets the constraint
        while max(returns_data['Cluster'].value_counts()) > max_stocks_per_cluster:
            num_clusters += 1
            clusters = fcluster(linkage_matrix, num_clusters, criterion='maxclust')
            returns_data['Cluster'] = clusters

        # Define the minimum number of stocks in a cluster
        minimum_stocks_in_cluster = 2

        # Filter out clusters with fewer than the minimum number of stocks
        filtered_clusters = returns_data.groupby('Cluster').filter(
            lambda x: len(x) >= minimum_stocks_in_cluster)['Cluster'].unique()

        # Assign the filtered cluster labels to the original price data
        returns_data = returns_data[returns_data['Cluster'].isin(filtered_clusters)]

        # Calculate the returns for each cluster and sum across clusters
        cluster_returns = returns_data.groupby('Cluster').mean().sum(axis=1)

        if portfolio == 'long-short':
            # Identify stocks to go short and long based on cluster returns
            short = np.array(returns_data[returns_data.Cluster ==
                                          cluster_returns.idxmin()].index)

            long = np.array(returns_data[returns_data.Cluster ==
                                         cluster_returns.idxmax()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go long and short
            long_returns = hold_returns[long].mean()
            short_returns = -1 * hold_returns[short].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the long and short portfolios for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                long) + list(short)][holding_month]

            # Adjust returns for short positions
            monthly_portfolio_returns[short] *= -1

        elif portfolio == 'long':
            long = np.array(returns_data[returns_data.Cluster ==
                                         cluster_returns.idxmax()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go long
            long_returns = hold_returns[long].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the long portfolio for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                long)][holding_month]

        elif portfolio == 'short':
            short = np.array(returns_data[returns_data.Cluster ==
                                          cluster_returns.idxmin()].index)

            # Extract the returns for holding stocks in the current month
            hold_returns = returns.iloc[-1]

            # Calculate the average returns for the stocks to go short
            short_returns = -1 * hold_returns[short].mean()

            # Copy monthly returns data for further manipulation
            returns_monthly = monthly_returns.copy()

            # Select returns for stocks in the short portfolio for the holding month
            monthly_portfolio_returns = returns_monthly[list(
                short)][holding_month]

        # Append adjusted returns for the holding month to the stock_monthly_returns dataframe
        stock_monthly_returns = stock_monthly_returns.append(monthly_portfolio_returns)

    return stock_monthly_returns

def plot_and_display_metrics_csmom(stock_monthly_returns):
    portfolio_returns = stock_monthly_returns.mean(axis=1)
    fig, ax = plt.subplots(figsize=(15, 7))
    portfolio_returns.plot(ax=ax)

    # Set the title and axis labels
    ax.set_title('Portfolio Returns Over Time')
    ax.set_xlabel('Time')
    ax.set_ylabel('Returns')
    ax.axhline(y=0, color='black', linestyle='-')

    # Fill area below 0 with red color
    ax.fill_between(portfolio_returns.index, portfolio_returns, 0,
                    where=portfolio_returns < 0, color='red', alpha=0.3)

    # Fill area above 0 with green color
    ax.fill_between(portfolio_returns.index, portfolio_returns, 0,
                    where=portfolio_returns >= 0, color='green', alpha=0.3)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')

    # Calculate cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    # Convert index to datetime format
    cumulative_returns.index = pd.to_datetime(cumulative_returns.index)

    # Plot cumulative returns
    plt.figure(figsize=(15, 7))
    cumulative_returns.plot()

    # Labeling axes and title
    plt.ylabel('Cumulative Returns', fontsize=12)
    plt.title('Cross Sectional Momentum Strategy Returns', fontsize=14)

    # Show plot
    plt.show()

    # Calculate maximum cumulative returns up to each point
    max_cumulative_returns = cumulative_returns.cummax()

    # Calculate drawdown
    drawdown = (cumulative_returns - max_cumulative_returns) / max_cumulative_returns

    # Plot drawdown
    plt.figure(figsize=(15, 7))
    # Fill area under the drawdown curve with red color
    plt.fill_between(drawdown.index, drawdown, 0, color='red', alpha=0.3)
    plt.ylabel('Drawdown', fontsize=12)
    plt.title('Cross Sectional Momentum Strategy Drawdown', fontsize=14)
    plt.show()

    # Display the metrics
    # Calculate monthly Sharpe ratio
    monthly_sharpe = portfolio_returns.mean() / portfolio_returns.std()

    # Annualize Sharpe ratio for monthly data
    sharpe = round(monthly_sharpe * ((12) ** 0.5),2)

    # Calculate drawdown
    max_cumulative_returns = cumulative_returns.cummax()
    drawdown = (cumulative_returns - max_cumulative_returns) / max_cumulative_returns
    max_drawdown_index = drawdown.idxmin()
    max_drawdown_date = max_drawdown_index.strftime('%Y-%m-%d')
    max_drawdown_value = round(drawdown.min(),2)

    # Create a DataFrame to hold the metrics
    metrics = pd.DataFrame({
        'Metric': ['Sharpe Ratio', 'Maximum Drawdown Date', 'Maximum Drawdown Value'],
        'Value': [sharpe, max_drawdown_date, max_drawdown_value]
    })

    # Display metrics
    print("\nPerformance Metrics:")
    display(metrics.rename_axis(None, axis=1))

# Data



## Features

In [ ]:
def get_parkinson(price_data, window=10, trading_periods=50, clean=True):
    rs = (1.0 / (4.0 * math.log(2.0))) * ((price_data['High'] / price_data['Low']).apply(np.log))**2.0

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(
        window=window,
        center=False
    ).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def get_HodgesTompkins(price_data, window=30, trading_periods=50, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    vol = log_return.rolling(
        window=window,
        center=False
    ).std() * math.sqrt(trading_periods)

    h = window
    n = (log_return.count() - h) + 1

    adj_factor = 1.0 / (1.0 - (h / n) + ((h**2 - 1) / (3 * n**2)))

    result = vol * adj_factor

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def get_skew(price_data, window=30, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    result = log_return.rolling(
        window=window,
        center=False
    ).skew()

    if clean:
        return result.dropna()
    else:
        return

In [ ]:
def get_kurtosis(price_data, window=30, clean=True):

    log_return = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)

    result = log_return.rolling(
        window=window,
        center=False
    ).kurt()

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def get_YangZhang(price_data, window=30, trading_periods=50, clean=True):

    log_ho = (price_data['High'] / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low'] / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    log_oc = (price_data['Open'] / price_data['Close'].shift(1)).apply(np.log)
    log_oc_sq = log_oc**2

    log_cc = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)
    log_cc_sq = log_cc**2

    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    close_vol = log_cc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))
    open_vol = log_oc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))
    window_rs = rs.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))

    k = 0.34 / (1.34 + (window + 1) / (window - 1))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * math.sqrt(trading_periods)

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def get_RogersSatchell(price_data, window=30, trading_periods=50, clean=True):

    log_ho = (price_data['High']  / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low']   / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(
        window=window,
        center=False
    ).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def get_GermanKlass(price_data, window=22, trading_periods=50, clean=True):

    log_hl = (price_data['High'] / price_data['Low']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)

    rs = 0.5 * log_hl**2 - (2*math.log(2)-1) * log_co**2

    def f(v):
        return (trading_periods * v.mean())**0.5

    result = rs.rolling(window=window, center=False).apply(func=f)

    if clean:
        return result.dropna()
    else:
        return result

In [ ]:
def create_features(stock_data: pd.DataFrame) -> pd.DataFrame:
    kalman_periods = [100, 300, 500, 700, 900]
    slope_length   = [3, 6, 9, 12, 15, 18, 21]

    features = pd.DataFrame(index=stock_data.index)

    # ── Kalman y derivados ─────────────────────────────────────────────
    for period in tqdm(kalman_periods, desc="Calculating Kalman and Derivatives"):
        kal = pd.Series(kalman_line(stock_data['Close'], kalman_length=period, smooth=3),
                        index=stock_data.index)

        features[f'Kal_{period}']        = kal
        features[f'Close_Kal_{period}']  = stock_data['Close'] - kal
        features[f'Kal_change_{period}'] = kal.diff()

    # ── Slopes ─────────────────────────────────────────────────────────
    for period in tqdm(kalman_periods, desc="Calculating Slopes"):
        for sLen in slope_length:
            df_s = slope(stock_data['Close'], length_kal=period, smooth_kal=3,
                         slopeLen=sLen, offset=-1)

            features[f'slope_div_{period}_{sLen}']             = df_s['slope_div']
            features[f'slope_signal_{period}_{sLen}']          = df_s['slope_signal']
            features[f'slope_angle_{period}_{sLen}']           = df_s['slope_angle']
            features[f'slope_angle_signal_{period}_{sLen}']    = df_s['slope_angle_signal']
            features[f'slope_lin_reg_{period}_{sLen}']         = df_s['slope_lin_reg']
            features[f'slope_lin_reg_signal_{period}_{sLen}']  = df_s['slope_lin_reg_signal']

    features.dropna(inplace=True)
    return features

## 1_min

In [ ]:
# Read the CSV file
df_1min = pd.read_csv(root_data + 'Data/'+symbol+'_M1.csv', index_col=0)
df_1min.index = pd.to_datetime(df_1min.index)
df_1min = df_1min.iloc[-500000:,]

print('Min_Date : ', df_1min.index.min())
print('Min_Date : ', df_1min.index.max())
print('Number_Rows = ',len(df_1min.index))
print('\n')

df_1min.tail()

**Features**

In [ ]:
features = create_features(df_1min)
features = features.dropna()
print("Number of features are:", features.shape[1])
print(features.shape)
features.to_csv(root_data+'Results/'+symbol+'_'+direction+'_M1_Raw_Features.csv')
features.tail(5)

In [ ]:
#for col in features.columns:
 # print(col)

In [ ]:
features = pd.read_csv(root_data+'Results/'+symbol+'_M1_Raw_Features.csv')
features["Date"] = pd.to_datetime(features["Date"])
print(features.shape)
features.head(5)

**Scale_features**

In [ ]:
cols_to_scale = features.columns[1:]
cols_to_scale

In [ ]:
### Scale Features with Rolling Window
window = 200
rolling = features[cols_to_scale].rolling(window)
features[cols_to_scale] = (features[cols_to_scale] - rolling.mean()) / rolling.std()

print("DataFrame with rolling-scaled features:")
print(features.shape)
features.head()


In [ ]:
features.to_csv(root_data+'Results/'+symbol+'_M1_Scale_Features.csv')

## 5_min

In [ ]:
# Read the CSV file
df_5min = pd.read_csv(root_data + 'Data/'+symbol+'_M5.csv', index_col=0)
df_5min.index = pd.to_datetime(df_5min.index)
df_5min = df_5min.iloc[-1000000:,]

print('Min_Date : ', df_5min.index.min())
print('Min_Date : ', df_5min.index.max())
print('Number_Rows = ',len(df_5min.index))
print('\n')

df_5min.tail()

**Features**

In [ ]:

### Create the Features Set
import time
start_time = time.time()

features = create_features(df_5min)
features = features.dropna()

end_time = time.time()
execution_time = end_time - start_time

print("Number of features are:", features.shape[1])
print(features.shape)
features.to_csv(root_data+'Results/'+symbol+'DRL_M5_Raw_Features.csv')
print(f"Execution time: {execution_time:.2f} seconds")
features.tail(5)

In [ ]:
for col in features.columns:
  print(col)

In [ ]:
features = pd.read_csv(root_data+'Results/'+symbol+'_M5_Raw_Features.csv')
features["Date"] = pd.to_datetime(features["Date"])
print(features.shape)
features.head(5)

**Scale_features**

In [ ]:
cols_to_scale = features.columns[1:]
cols_to_scale

In [ ]:
### Scale Features with Rolling Window
start_time = time.time()

window = 200
rolling = features[cols_to_scale].rolling(window)
features[cols_to_scale] = (features[cols_to_scale] - rolling.mean()) / rolling.std()

print("DataFrame with rolling-scaled features:")
print(features.shape)
features.head()
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")


In [ ]:
features.to_csv(root_data+'Results/'+symbol+'DRL_M5_Scale_Features.csv')

## 10_min

In [ ]:
# Read the CSV file
df_10min = pd.read_csv(root_data + 'Data/'+symbol+'_M10.csv', index_col=0)
df_10min.index = pd.to_datetime(df_10min.index)
df_10min = df_10min.iloc[-100000:,]

print('Min_Date : ', df_10min.index.min())
print('Min_Date : ', df_10min.index.max())
print('Number_Rows = ',len(df_10min.index))
print('\n')

df_10min.tail()

**Features**

In [ ]:
### Create the Features Set
features_10min = create_features(df_10min)
features_10min = features_10min.dropna()

# Add "10min_" prefix to all column names except 'Date'
features_10min = features_10min.add_prefix('10min_')
features_10min.rename(columns={'10min_Date': 'Date'}, inplace=True)

print("Number of features are:", features.shape[1])
print(features_10min.shape)
features_10min.to_csv(root_data+'Results/'+symbol+'_M10_Raw_Features.csv')
features_10min.tail(5)

In [ ]:
for col in features_10min.columns:
  #print(col)

In [ ]:
features_10min = pd.read_csv(root_data+'Results/'+symbol+'_M10_Raw_Features.csv')
features_10min["Date"] = pd.to_datetime(features_10min["Date"])
print(features_10min.shape)
features_10min.head(5)

**Scale_features**

In [ ]:
cols_to_scale = features_10min.columns[1:]
cols_to_scale

In [ ]:
### Scale Features with Rolling Window
window = 200
rolling = features_10min[cols_to_scale].rolling(window)
features_10min[cols_to_scale] = (features_10min[cols_to_scale] - rolling.mean()) / rolling.std()

print("DataFrame with rolling-scaled features:")
print(features_10min.shape)
features_10min.tail()


In [ ]:
features_10min.to_csv(root_data+'Results/'+symbol+'_M10_Scale_Features.csv')

# Feature Importance

## Labels

In [ ]:
lab = pd.read_csv(root_data + 'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Strategy_Gen_Labels.csv', index_col=0)
lab['Date'] = pd.to_datetime(lab['Date'])

print('Min_Date    : ',lab['Date'].min())
print('Min_Date    : ',lab['Date'].max(),'\n')
print('Number_Rows : ',lab.shape,'\n')
print('Columns     : ',lab.columns)

lab['Open_Trade'].value_counts()

In [ ]:
st_max_1 = lab.loc[((lab[result_field] > 0) & (lab['st_Max'] > 0)),'st_atr_max_PnL'].describe()
st_max_2 = lab.loc[((lab[result_field] < 0) & (lab['st_Max'] > 0)),'st_atr_max_PnL'].describe()
st_max_3 = lab.loc[((lab[result_field] < 0) & (lab['st_Max'] <= 0)),'st_atr_PnL'].describe()
st_max_4 = lab.loc[((lab[result_field] < 0)),'st_atr_PnL'].describe()

combined_descriptions = pd.concat([st_max_1, st_max_2, st_max_3, st_max_4], axis=1)
combined_descriptions.columns = ['st_max_1', 'st_max_2', 'st_max_3', 'st_max_4']

print("--- Descriptive Statistics when Open_Trade is -1 (Side-by-Side) ---")
display(combined_descriptions)

In [ ]:
#analyse_column = 'st_atr_max_PnL'
analyse_column = 'st_Max'

st_max_0  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),:]['Open_Trade'].count()
st_max_1  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),analyse_column].mean()
st_max_2  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] >= 1.93),analyse_column].sum()
st_max_25 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 1.93),analyse_column].sum()


st_max_3 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 0)),analyse_column].sum()

st_max_4 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 0.7) & (lab['st_atr_max_PnL'] <= 1)),analyse_column].sum()

st_max_5 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 1) &
                   (lab['st_atr_max_PnL'] <= 1.5)),analyse_column].sum()

st_max_6 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 1.5) &
                   (lab['st_atr_max_PnL'] <= 2)),analyse_column].sum()

st_max_7 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 2)),analyse_column].sum()


print(f'Total_Trades = {st_max_0:,.0f}\n')
print(f'Mean st_atr_max_PnL = {st_max_1:,.2f}\n')
print(f'Above_Mean = {st_max_2:,.2f}')
print(f'Below_Mean = {st_max_25:,.2f}\n')

print(f'<= 0.5 = {st_max_3:,.2f}')
print(f'> 0.5 & <= 1 = {st_max_4:,.2f}')
print(f'> 1 & <= 1.5 = {st_max_5:,.2f}')
print(f'> 1.5 & <= 2 = {st_max_6:,.2f}')
print(f'> 2 = {st_max_7:,.2f}')

In [ ]:
analyse_column = 'st_atr_max_PnL'

st_max_0 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),:]['Open_Trade'].count()
st_max_1 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),analyse_column].mean()
st_max_2 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] >= 1.93),analyse_column].count()
st_max_25 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 1.93),analyse_column].count()

st_max_3 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                    (lab['st_atr_max_PnL'] <= 0.5)),analyse_column].count()

st_max_4 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                     (lab['st_atr_max_PnL'] >= 0.5) & (lab['st_atr_max_PnL'] <= 1)),analyse_column].count()

st_max_5 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] >= 1), analyse_column].count()

print(f'Total_Trades = {st_max_0:,.0f}\n')
print(f'Mean st_atr_max_PnL = {st_max_1:,.2f}\n')
print(f'Above_Mean = {st_max_2:,.2f}')
print(f'Below_Mean = {st_max_25:,.2f}\n')

print(f'<= 0.5 = {st_max_3:,.2f}')
print(f'> 0.5 & <= 1 = {st_max_4:,.2f}')
print(f'> 1 = {st_max_5:,.2f}')


In [ ]:
# --- Parámetros / campos
result_field = 'st_atr_max_PnL'

valid = (
    (lab['Type'] == direction) &
    (lab['Open_Trade'].isin([1, -1])) &
    (lab[result_field].notna())
)

# --- Conteos por rango (st_max_4..6)
st_max_4 = (valid & (lab[result_field] <= 0.5)).sum()
st_max_5 = (valid & (lab[result_field] >= 0.5) & (lab[result_field] <= 1.0)).sum()
st_max_6 = (valid & (lab[result_field] > 1.0)).sum()

print(f'<= 0.5          = {st_max_4:,d}')
print(f'> 0.5 & <= 1.0  = {st_max_5:,d}')
print(f'> 1.0           = {st_max_6:,d}')

# --- Etiquetado en la columna "label" con valores 4/5/6
lab['label'] = np.nan
lab.loc[valid & (lab[result_field] <= 0.5), 'label'] = 0
lab.loc[valid & (lab[result_field] > 0.5) & (lab[result_field] <= 1.0), 'label'] = 1
lab.loc[valid & (lab[result_field] > 1.0), 'label'] = 2

# --- Mantener solo filas válidas y con label
lab = lab.loc[valid & lab['label'].notna()].copy()
lab['label'] = lab['label'].astype('int8')

# --- Ver distribución de labels 4/5/6
print('\nValue counts de label 4/5/6:')
print(lab['label'].value_counts(dropna=False).sort_index())


In [ ]:
lab['expected'] = np.where(lab['label']==0, -100,
                           np.where(lab['label']==1, 50,
                                    np.where(lab['label']==2,lab[result_field],np.nan)))
lab['expected'].sum()


## Features

In [ ]:
raw_feat_1min = pd.read_csv(root_data+'Results/'+symbol+'_M1_Raw_Features.csv')
raw_feat_1min["Date"] = pd.to_datetime(raw_feat_1min["Date"])
print(raw_feat_1min.shape)
#raw_feat_1min.head(5)

In [ ]:
scale_feat_1min = pd.read_csv(root_data+'Results/'+symbol+'_M1_Scale_Features.csv')
scale_feat_1min = scale_feat_1min.drop('Unnamed: 0', axis=1)
scale_feat_1min["Date"] = pd.to_datetime(scale_feat_1min["Date"])
print(scale_feat_1min.shape)
#scale_feat_5min.head(5)

In [ ]:
raw_feat_5min = pd.read_csv(root_data+'Results/'+symbol+'_M5_Raw_Features.csv')
raw_feat_5min["Date"] = pd.to_datetime(raw_feat_5min["Date"])
print(raw_feat_5min.shape)
#raw_feat_5min.head(5)

In [ ]:
scale_feat_5min = pd.read_csv(root_data+'Results/'+symbol+'_M5_Scale_Features.csv')
scale_feat_5min = scale_feat_5min.drop('Unnamed: 0', axis=1)
scale_feat_5min["Date"] = pd.to_datetime(scale_feat_5min["Date"])
print(scale_feat_5min.shape)
#scale_feat_5min.head(5)

In [ ]:
raw_feat_10min = pd.read_csv(root_data+'Results/'+symbol+'_M10_Raw_Features.csv')
raw_feat_10min["Date"] = pd.to_datetime(raw_feat_10min["Date"])

# Add "10min_" prefix to all column names except 'Date'
cols_to_prefix = [col for col in raw_feat_10min.columns if col != 'Date']
raw_feat_10min.rename(columns={col: col for col in cols_to_prefix}, inplace=True)

print(raw_feat_10min.shape)
#raw_feat_10min.head(5)

In [ ]:
scale_feat_10min = pd.read_csv(root_data+'Results/'+symbol+'_M10_Scale_Features.csv')
scale_feat_10min = scale_feat_10min.drop('Unnamed: 0', axis=1)
scale_feat_10min["Date"] = pd.to_datetime(scale_feat_10min["Date"])

# Add "10min_" prefix to all column names except 'Date'
cols_to_prefix = [col for col in scale_feat_10min.columns if col != 'Date']
scale_feat_10min.rename(columns={col:col for col in cols_to_prefix}, inplace=True)

print(scale_feat_10min.shape)
#scale_feat_10min.head(5)

## Merge

In [ ]:
data_type = 'Scale'

In [ ]:
# First merge scale_feat_5min with scale_feat_10min and apply forward fill
merged_features = scale_feat_5min.merge(scale_feat_10min, on='Date', how='left').ffill()
#merged_features = raw_feat_5min.merge(raw_feat_10min, on='Date', how='left').ffill()

# Then merge with the 'label' column from lab using the lab rows and 'Date' as key
df = pd.merge(lab[['Date', 'label']], merged_features, on='Date', how='left')

cols = df.columns.tolist()
cols.remove('label')
cols.insert(1, 'label')
df = df[cols]

In [ ]:
print(df.columns, '\n')
print(df.shape,'\n')
print('Label_Counts : ',df.label.value_counts(),'\n')
#df.head(5)

## ML

In [ ]:
# ===================== 1. ENTRENAR Y OBTENER IMPORTANCIAS =====================
def compute_xgb_importance(
    X: pd.DataFrame,
    y: pd.Series,
    task: str = "classification",
    random_state: int = 42,
    **xgb_params: Any
) -> Tuple[pd.DataFrame, Any]:
    """
    Entrena un modelo XGBoost y devuelve:
      - imp_df: DataFrame con 'feature', 'importance' y 'cum_importance'.
      - model : modelo ya entrenado.

    Soporta:
      • Clasificación binaria o multiclase (detecta nº de clases).
      • Regresión (si task != 'classification').

    Parámetros
    ----------
    X : pd.DataFrame
        Matriz de características (sin la columna objetivo).
    y : pd.Series
        Etiquetas objetivo. Puede ser binaria (0/1) o multiclase (0..K-1).
    task : str, opcional
        "classification" (default) o "regression".
    random_state : int, opcional
        Semilla para reproducibilidad.
    **xgb_params : dict
        Parámetros adicionales para el estimador de XGBoost.

    Returns
    -------
    (imp_df, model)
        imp_df : DataFrame con importancias y su acumulado.
        model  : instancia entrenada de XGBClassifier / XGBRegressor.
    """
    default_params: Dict[str, Any] = dict(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=random_state,
        n_jobs=-1,
        tree_method="hist",
    )
    default_params.update(xgb_params)

    if task == "classification":
        # Detectar nº de clases
        classes = np.unique(y)
        n_classes = len(classes)

        # XGBClassifier ajusta objetivo automáticamente, pero lo explicitamos:
        if n_classes > 2:
            default_params.setdefault("objective", "multi:softprob")
            default_params.setdefault("num_class", n_classes)
            eval_metric = "mlogloss"
        else:
            default_params.setdefault("objective", "binary:logistic")
            eval_metric = "logloss"

        model = XGBClassifier(eval_metric=eval_metric, **default_params)

    else:
        model = XGBRegressor(**default_params)

    model.fit(X, y)

    imp_df = (
        pd.DataFrame({
            "feature": X.columns,
            "importance": model.feature_importances_
        })
        .sort_values("importance", ascending=False)
        .reset_index(drop=True)
    )
    total_imp = imp_df["importance"].sum()
    if total_imp == 0:
        # Evitar división por cero si el modelo devuelve todo cero (raro, pero posible)
        imp_df["cum_importance"] = 0.0
    else:
        imp_df["cum_importance"] = imp_df["importance"].cumsum() / total_imp

    return imp_df, model

In [ ]:
# ===================== 2. SELECCIÓN DE FEATURES =====================
def select_features_with_importance(
    X: pd.DataFrame,
    imp_df: pd.DataFrame,
    top_n: Optional[int] = None,
    threshold: Optional[str | float] = None,
    cum_threshold: Optional[float] = 0.8
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Selección flexible de variables a partir de importancias de XGBoost.

    Reglas:
      - Si top_n no es None           => usa el top_n.
      - Else si cum_threshold no None => usa importancia acumulada (p.ej. 0.8 = 80%).
      - Else usa threshold ('median', 'mean' o valor numérico).

    Devuelve (X_reducido, lista_de_features).

    Parámetros
    ----------
    X : pd.DataFrame
        Matriz de características original.
    imp_df : pd.DataFrame
        DataFrame devuelto por compute_xgb_importance.
    top_n : int | None
        Número fijo de variables a conservar.
    threshold : str | float | None
        Umbral de importancia. Si str, usar 'median' o 'mean'.
    cum_threshold : float | None
        Porcentaje acumulado de importancia (0-1). Si None, se ignora.

    Returns
    -------
    (X_sel, keep)
        X_sel : subset de X con columnas seleccionadas.
        keep  : lista de nombres de columnas seleccionadas.
    """
    if top_n is not None:
        keep = imp_df.head(top_n)["feature"].tolist()

    elif cum_threshold is not None:
        keep_mask = imp_df["cum_importance"] <= float(cum_threshold)
        keep = imp_df.loc[keep_mask, "feature"].tolist()
        # asegurar que haya al menos una más para no quedarnos exactamente en el corte
        if len(keep) < len(imp_df):
            keep.append(imp_df.iloc[len(keep)]["feature"])

    else:
        if threshold is None:
            threshold = "median"
        if isinstance(threshold, str):
            thr_val = imp_df["importance"].agg(threshold)
        else:
            thr_val = float(threshold)
        keep = imp_df.loc[imp_df["importance"] >= thr_val, "feature"].tolist()

    return X[keep], keep

In [ ]:
# ===================== 3. BÚSQUEDA DEL MEJOR UMBRAL ACUMULADO =====================
def find_best_cum_threshold(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_valid: pd.DataFrame,
    y_valid: pd.Series,
    task: str = "classification",
    thresholds: Tuple[float, ...] = (0.6, 0.7, 0.8, 0.9),
    random_state: int = 42,
    metric: str = "auto",
    **xgb_params: Any
) -> Tuple[float, pd.DataFrame, pd.DataFrame]:
    """
    Entrena un XGB en train, calcula importancias y prueba varios umbrales
    acumulados para ver cuál da la mejor métrica en valid.

    Para CLASIFICACIÓN:
        - Detecta nº de clases.
        - Métrica por defecto (metric="auto"):
            • Binaria: ROC-AUC (probabilidades de la clase positiva).
            • Multiclase: ROC-AUC macro OVR (usa predict_proba).
          Alternativas: metric="f1_macro", "accuracy", "logloss" (se MINIMIZA).
    Para REGRESIÓN:
        - Usa R^2.

    Devuelve:
        best_thr, res_df_ordenado_por_score_desc, imp_df

    Parámetros
    ----------
    X_train, y_train, X_valid, y_valid : pd.DataFrame / pd.Series
        Particiones de entrenamiento y validación.
    task : str
        "classification" (default) o "regression".
    thresholds : tuple[float, ...]
        Valores de umbral de importancia acumulada a evaluar (0-1).
    random_state : int
        Semilla para reproducibilidad.
    metric : str
        "auto" (default), "roc_auc", "f1_macro", "accuracy", "logloss" (clasif) o "r2" (regresión).
    **xgb_params : dict
        Parámetros extra para el estimador de XGBoost (pasan a compute y a los modelos internos).

    Returns
    -------
    (best_thr, res_df, imp_df)
        best_thr : float
            Umbral con mejor score (o menor logloss si metric='logloss').
        res_df : pd.DataFrame
            Tabla con resultados por umbral (n_features, score).
        imp_df : pd.DataFrame
            Importancias calculadas en X_train / y_train.
    """
    imp_df, _ = compute_xgb_importance(
        X_train, y_train, task=task, random_state=random_state, **xgb_params
    )

    results = []

    # Detectar nº de clases si es clasificación
    if task == "classification":
        classes = np.unique(y_train)
        n_classes = len(classes)
        if metric == "auto":
            metric_to_use = "roc_auc" if n_classes == 2 else "roc_auc"
        else:
            metric_to_use = metric
    else:
        metric_to_use = "r2" if metric == "auto" else metric

    for thr in thresholds:
        X_tr_sel, cols = select_features_with_importance(
            X_train, imp_df, cum_threshold=thr, top_n=None, threshold=None
        )
        X_va_sel = X_valid[cols]

        if task == "classification":
            params = dict(random_state=random_state, n_jobs=-1, tree_method="hist")
            params.update(xgb_params)

            if n_classes > 2:
                params.setdefault("objective", "multi:softprob")
                params.setdefault("num_class", n_classes)
                eval_metric = "mlogloss"
            else:
                params.setdefault("objective", "binary:logistic")
                eval_metric = "logloss"

            model_sel = XGBClassifier(eval_metric=eval_metric, **params)
            model_sel.fit(X_tr_sel, y_train)

            # Probabilidades y predicciones
            proba = model_sel.predict_proba(X_va_sel)
            pred  = np.argmax(proba, axis=1) if n_classes > 2 else (proba[:, 1] >= 0.5).astype(int)

            # Calcular métrica
            if metric_to_use == "roc_auc":
                if n_classes == 2:
                    score = roc_auc_score(y_valid, proba[:, 1])
                else:
                    # AUC macro One-vs-Rest
                    score = roc_auc_score(y_valid, proba, multi_class="ovr", average="macro")
            elif metric_to_use == "f1_macro":
                score = f1_score(y_valid, pred, average="macro")
            elif metric_to_use == "accuracy":
                score = accuracy_score(y_valid, pred)
            elif metric_to_use == "logloss":
                # En este caso, menor es mejor. Guardamos negativo para mantener criterio "mayor mejor".
                score = -log_loss(y_valid, proba, labels=np.unique(y_train))
            else:
                raise ValueError(f"Métrica no soportada: {metric_to_use}")

        else:
            # REGRESIÓN
            params = dict(random_state=random_state, n_jobs=-1, tree_method="hist")
            params.update(xgb_params)
            model_sel = XGBRegressor(**params)
            model_sel.fit(X_tr_sel, y_train)
            pred = model_sel.predict(X_va_sel)

            if metric_to_use == "r2":
                score = r2_score(y_valid, pred)
            else:
                raise ValueError(f"Métrica de regresión no soportada: {metric_to_use}")

        results.append({"cum_threshold": thr, "n_features": len(cols), "score": score})

    # Ordenar (si usamos logloss negado, mayor sigue siendo mejor)
    res_df = pd.DataFrame(results).sort_values("score", ascending=False).reset_index(drop=True)
    best_thr = float(res_df.iloc[0]["cum_threshold"])
    return best_thr, res_df, imp_df

In [ ]:
# ===================== 3. PIPELINE PRINCIPAL =====================
df = df.dropna()
y = df["label"]
X = df.iloc[:, 2:]

# --- 3.3 Split temporal (ejemplo simple 80/20) ---
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# --- 3.4 Importancias con XGBoost ---
imp_df, xgb_model = compute_xgb_importance(X_train, y_train, task="classification")

print("=== Importancias XGBoost ===")
print(imp_df.head(20))
print(f"Total features: {len(imp_df)}")

# --- 3.5 Selección (elige una opción) ---
X_train_sel, keep_cols = select_features_with_importance(X_train, imp_df, cum_threshold=0.8)
X_test_sel = X_test[keep_cols]

print(f"Features seleccionadas: {len(keep_cols)}")
pd.Series(keep_cols).to_csv(root_data+'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_ImportantCols.csv', index=False)

In [ ]:
keep_cols.insert(0, 'Date')
#keep_cols

In [ ]:
df = df.loc[:,keep_cols]
#df.head(5)

In [ ]:
df.to_csv(root_data+'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_Features.csv')

# Encode_Features

In [ ]:
### Encode Features

cols_to_scale = df.columns[1:]
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(len(cols_to_scale),)))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(cols_to_scale), activation='linear'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder using the scaled features
cols_to_scale = df.columns[1:]
autoencoder = model.fit(df[cols_to_scale], df[cols_to_scale], epochs=100, batch_size=32)

In [ ]:
# Plot the graph of Loss versus Epoch
plt.plot(autoencoder.history["loss"])
plt.plot(figsize=(15, 7))
plt.title('Loss vs. Epoch', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.show()

In [ ]:
# Select the columns to be scaled (from index 6 onwards)
cols_to_scale = df.columns[1:]

reconstruction_error = np.square(df[cols_to_scale] - model.predict(df[cols_to_scale]))
feature_reconstruction_error = np.mean(reconstruction_error, axis=0)
feature_reconstruction_error_df = pd.DataFrame(feature_reconstruction_error, index=cols_to_scale).T

overall_reconstruction_error = feature_reconstruction_error_df.mean().mean()
print('\n','\n', f"\033[1mOverall Reconstruction Error: {overall_reconstruction_error:.4f}\033[0m", '\n','\n')

# Print the individual features error in a horizontal format
display(feature_reconstruction_error_df)

You can see that certain features have very low error, but some might have an error as high as 0.2. However, overall the error rate is 0.04 and thus, we can move forward.

Let us now move towards finding the reduced features. This is done in the following steps.

**Extract the encoder part of the autoencoder**:
You will extract the encoder part of the autoencoder, which compresses the data into a reduced-dimensional representation. This part includes the first three layers of the model.

Use the encoder to obtain the reduced-dimensional representation:
We use the encoder to transform the input test data `(X_test)` into a reduced-dimensional representation `(X_encoded_test)`. This represents an encoded version of the input data.

In [ ]:
# Re-create the encoder from the layers of the successfully trained model
encoder = Sequential(model.layers[:3])
X_encoded_test = encoder.predict(X[cols_to_scale]) # Use the scaled features for prediction

In [ ]:

# ── NUEVO: añade la columna Date al DataFrame codificado ──
dates = df['Date'].reset_index(drop=True)                 # 1) copia la fecha
features_enc = pd.DataFrame(                              # 2) crea el DF codificado
    X_encoded_test,
    columns=[f'Encoded_{i}' for i in range(X_encoded_test.shape[1])]
)
features_enc.insert(0, 'Date', dates)
features_enc.head(5)

In [ ]:
features_enc.shape

In [ ]:
### Save the encoder model

encoder_save_path = root_data+'Models/'+symbol+'_'+direction+'_'+data_type+'_ahm_encoder_model.keras'
os.makedirs(os.path.dirname(encoder_save_path), exist_ok=True)
encoder.save(encoder_save_path)
print(f"Encoder model saved successfully at: {encoder_save_path}")

In [ ]:
features_enc.to_csv(root_data+'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_Enc_Features.csv')

# Varios

In [ ]:
df.tail(5)